# Feature Extraction

The dataset provided contains CT-scans of different lung patients. In order to extract some features out of these, we used two different python libraries, pylidc and pyradiomics. Before merging the two, we also turned our target variable, which took values from 1 to 5, into a binary variable (benign vs malign).

We start by importing relevant libraries.

In [ ]:
import pylidc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

### PyLIDC

We create a dataframe to store all the information we aim to extract

In [ ]:
df_pylidc = pd.DataFrame(columns= [
                            'patient_id', 
                            'annotation_id',
                            'scan_id',
                            
                            'slice_thickness',
                            'pixel_spacing',

                            'subtlety', 
                            'internalStructure', 
                            'calcification', 
                            'sphericity', 
                            'margin', 
                            'lobulation', 
                            'spiculation', 
                            'texture', 
                        
                            'diameter',
                            'surface_area',
                            'volume',

                            'malignancy',
                            ])

And then we estract the data and save it to a CSV file.

In [ ]:
ann = pylidc.query(pylidc.Annotation).all()

for i in range(len(ann)):
    att  = dict((col, "") for col in df_pylidc.columns)

    # patient and annotation identification
    att['patient_id'] = ann[i].scan.patient_id
    att['annotation_id'] = ann[i].id  
    att['scan_id'] = ann[i].scan.id

    # features
    st = pylidc.query(pylidc.Scan.slice_thickness).filter(pylidc.Scan.id == att['scan_id'])
    s = str(st[0])
    att['slice_thickness'] = float(s[1:4])
    ps = pylidc.query(pylidc.Scan.pixel_spacing).filter(pylidc.Scan.id == att['scan_id'])
    p = str(ps[0])
    att['pixel_spacing'] = float(p[1:5])

    att['subtlety'] = ann[i].subtlety
    att['internalStructure'] = ann[i].internalStructure 
    att['calcification'] = ann[i].calcification 
    att['sphericity'] = ann[i].sphericity
    att['margin'] = ann[i].margin  
    att['lobulation'] = ann[i].lobulation
    att['spiculation'] = ann[i].spiculation 
    att['texture'] = ann[i].texture

    att['diameter'] = ann[i].diameter
    att['surface_area'] = ann[i].surface_area
    att['volume'] = ann[i].volume

    # target
    att['malignancy'] = ann[i].malignancy  

    df_pylidc = df_pylidc.append(att, ignore_index=True)

df_pylidc.to_csv('pylidc_features.csv', sep=',', index=False)

#### Exploring the data attained

In [ ]:
df_pylidc = pd.read_csv('pylidc_features.csv', index_col=False)
df_pylidc.head()

In [ ]:
# number of individual patients assessed in the data
patients = np.unique(df_pylidc.patient_id)
print(f"Total number of patients: {len(patients)} \n")

df_pylidc.info()

#### Standardizing Indexes

We start by sorting the dataframe by 'patient_id' and 'annotation_id' and grouping it by 'patient_id' in a new index column.

In [ ]:
df_pylidc.sort_values(by=['patient_id', 'annotation_id'], inplace=True)

df_pylidc['Ann_id'] = df_pylidc.groupby('patient_id').cumcount() + 1
df_pylidc = df_pylidc[['Ann_id'] + [col for col in df_pylidc.columns if col != 'Ann_id']]

We then format the new ID to match with data we will get from PyRadiomics and drop the redundant columns, saving the dataframe to a new CSV file.

In [ ]:
for i in range(len(df_pylidc)):
    df_pylidc.at[i,'Id'] = df_pylidc.at[i,'patient_id'] + '-' + str(df_pylidc.at[i,'Ann_id'])

df_pylidc = df_pylidc[['Id'] + [col for col in df_pylidc.columns if col != 'Id']]
df_pylidc = df_pylidc.drop(columns=['Ann_id', 'annotation_id', 'scan_id'])

df_pylidc.to_csv('pylidc_features_fixed.csv', sep=',', index=False)

### PyRadiomics

Pyradiomics doesn't deal with DICOM files directly. In their documentation, though, it is possible to find a experimental script (pyradiomics-dcm.py) that supports its use with DICOM data, using plastimatch and dcmqi.

We modified this script to append new lines to an already existing, but containing only the headers, CSV file and used it for our feature extraction.

This script was given as input:
- a directory with the input DICOM series
- the file name pointing to a DICOM Segmentation Image (DICOM SEG) object
- the file name pointing to a yaml with the desired extraction parameters
- the file name pointing to a tsv dictionary that maps pyradiomics feature names to the IBSI defined features
- the ID for that segmentation

Like this, we only needed to iterate through all of the patients and respective segmentations to process the extraction.

In [ ]:
main_directory = os.listdir()

for patient in main_directory:
    # enter each patients folder, ignoring the folders for storing output and other files that may be inside the main directory
    if (not os.path.isdir(patient)) or patient == "OutputSR" or patient == "TempDir":
        continue
    
    folders = os.listdir(patient) # list of folders inside patient (CT-scan and X-ray)
    path = ""
    content = 0
    
    # finding the CT-scan folder (has the biggest content - directories with each segmentation and directory with the input DICOM series)
    for folder in folders:
        segmentations_or_series = os.listdir(patient + "\\" + folder)
        if (len(segmentations_or_series) > content):
            content = len(segmentations_or_series)
            path = folder
    
    folders = os.listdir(patient + "\\" + path)
    main = ""

    # finding the series folder
    for folder in folders:
        if not "Annotation" in folder:
            main = folder
            break
        
    seg_index = 1
    
    # extract features for each segmentation and naming each line with Patient_Name-Segmentation_Number
    for folder in folders:
        if "Segmentation" in folder:
            print("\n\nPACIENTE NUMERO " + patient + " - SEGMENTAÇÃO " + str(seg_index))
            os.system(f'cmd /c """python pyradiomics-dcm.py --input-image-dir "{patient}\{path}\{main}" --input-seg-file "{patient}\{path}\{folder}\\1-1.dcm" --output-dir OutputSR --temp-dir TempDir --parameters Pyradiomics_Params.yaml --features-dict featuresDict.tsv --name {patient}-{seg_index}"""')
            seg_index+=1


The previous script takes days to run for all patients, which is why we found it unwise to run it again inside the notebook.

The attained CSV file didn't process some of the features due to some images being too small to apply the Log filter and didn't process some segmentations due to the mask having too few dimensions or voxels.

Regardless, our final Pyradiomics CSV was as following.

#### Exploring the data attained

In [ ]:
df_pyradiomics = pd.read_csv('pyradiomics_features.csv', index_col=False)
df_pyradiomics.info()

In [ ]:
# number of individual patients assessed in the data
patients = np.unique(df_pyradiomics.id)
print(f"Total number of patients: {len(patients)} \n")

df_pyradiomics.info()

Looking at the whole dataframe, we were able to see that the lines where the Log Filter wasn't correctly applied were misplaced.

#### Fixing misplaced cells

In [ ]:
df_pyradiomics.iloc[:, 143:] = df.iloc[:, 143:].apply(lambda x: x.shift(273) if x.isna().any() else x, axis=1)
df.to_csv('pyradiomics_extraction_fixed.csv', index=False)

Finally, we have the result of both feature extractions.